### Bike_Sharing_Demand - Forecast use of a city bikeshare system

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv('共享单车 train.csv') # 我的文件的名字
X_train, X_test, y_train, y_test = exam_data_load(df, target='count')#, id_name='Id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8708, 11), (2178, 11), (8708, 2), (2178, 2))

In [47]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8708, 11), (2178, 11), (8708, 2), (2178, 2))

In [48]:
from sklearn.metrics import accuracy_score, mean_squared_error
dt = DecisionTreeRegressor()
dt.fit(X_tr,y_tr)
y_pred = dt.predict(X_ts)
print(dt.score(X_tr,y_tr))
print(round(np.sqrt(mean_squared_error(y_ts,y_pred)),3))

0.9942778268035373
165.589
